<a href="https://colab.research.google.com/github/Soban-Saleem/Agentic-and-Robotic-AI-Engineer/blob/main/LangChain_Gemini_Tool_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


langchain: For tool integration and conversational chains.
google-generativeai: To work with the Google Gemini Flash model.



In [45]:
!pip install -Uq langchain-google-genai


In [14]:
!pip install -qU langchain google-generativeai python-dotenv

Loading Google Gemini Flash API Key Securely


In [15]:
import os
from google.colab import userdata
import google.generativeai as genai


os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_1')

Defining Custom TOOLs

In [76]:
def add_numbers(params) -> float:
    """Adds two numbers together."""
    print(f"add_numbers called with params: {params}")
    if isinstance(params,str):
      params = json.loads(params)
    return params["a"] + params["b"]

def subtract_numbers(params) -> float:
    """Subtracts two numbers."""
    print(f"subtract_numbers called with params: {params}")
    if isinstance(params,str):
      params = json.loads(params)
    return params["a"] - params["b"]

def multiply_numbers(params) -> float:
    """Multiplies two numbers."""
    print(f"multiply_numbers called with params: {params}")
    if isinstance(params,str):
      params = json.loads(params)
    return params["a"] * params["b"]

def divide_numbers(params) -> float:
    """Divides two numbers."""
    print(f"divide_numbers called with params: {params}")
    if isinstance(params,str):
      params = json.loads(params)
    if params["b"] == 0:
        return "Cannot divide by zero."
    return params["a"] / params["b"]

Creating Tool Wraper for Langchain

In [29]:
from langchain.tools import Tool

In [30]:
tools = [
    Tool(
        name="add_numbers",
        func=add_numbers,
        description="Useful for when you need to add two numbers together. The input should be a dictionary with keys 'a' and 'b'.",
    ),
    Tool(
        name="subtract_numbers",
        func=subtract_numbers,
        description="Useful for when you need to subtract one number from another. The input should be a dictionary with keys 'a' and 'b'.",
    ),
   Tool(
        name="multiply_numbers",
        func=multiply_numbers,
        description="Useful for when you need to multiply two numbers. The input should be a dictionary with keys 'a' and 'b'.",
    ),
    Tool(
        name="divide_numbers",
        func=divide_numbers,
        description="Useful for when you need to divide one number by another. The input should be a dictionary with keys 'a' and 'b'.",
    ),
]

Setting up the Google Gemini Flash Model

In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI

initializing model

In [32]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp",
convert_system_message_to_human=True)

binding tools

In [41]:
llm_with_tools = llm.bind_tools(tools)


In [49]:
from IPython.display import display



Coversational Chain

In [58]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00


In [62]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.schema import SystemMessage
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage
from langchain.agents import AgentExecutor
import json
from langchain.tools import Tool


prompt

In [64]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            "You are a helpful assistant that can use tools to answer the users questions."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
        ("user", "{input}"),
    ]
)



tool calling chain


In [66]:
agent = create_tool_calling_agent(llm, tools, prompt)
chain = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [89]:
chat_history = ChatMessageHistory()
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    response = chain.invoke(
        {"input": user_input, "chat_history": chat_history.messages}
    )
    chat_history.add_user_message(user_input)
    if "intermediate_steps" in response and response["intermediate_steps"] and response["intermediate_steps"][-1][0].tool_calls:
      tool_calls = response["intermediate_steps"][-1][0].tool_calls
      for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = tool_call.function.arguments
        if isinstance(tool_args, str):
            tool_args = json.loads(tool_args)
        tool_output = tools[
            [tool.name for tool in tools].index(tool_name)
        ].func(tool_args)
        print(f"Tool Output: {tool_output}")
        chat_history.add_message(AIMessage(content=f"Tool {tool_name} returned: {tool_output}"))
        response = chain.invoke(
            {"input": user_input, "chat_history": chat_history.messages}
        )
    print(f"AI: {response['output']}")
    chat_history.add_ai_message(response['output'])

You: 8  divided by 5


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `divide_numbers` with `{"a": 8, "b": 5}`


divide_numbers called with params: {"a": 8, "b": 5}
1.6

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


That is 1.6.

> Finished chain.
AI: That is 1.6.
You: 9 multiply by 3


> Entering new AgentExecutor chain...


...

9 multiplied by 3 is 27.

> Finished chain.
AI: 9 multiplied by 3 is 27.
You: can you tell me 7 times of 5


> Entering new AgentExecutor chain...


/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Invoking: `multiply_numbers` with `{"a": 7, "b": 5}`


multiply_numbers called with params: {"a": 7, "b": 5}
35

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


7 times 5 is 35.

> Finished chain.
AI: 7 times 5 is 35.
You: exit
